# Hello World: Quantum Machine Learning with Merlin

Welcome! This notebook demonstrates how to use Merlin to build, train, and evaluate a hybrid quantum-classical neural network for the classic Iris classification task.  
You'll see how quantum layers can be integrated into standard PyTorch models and how to compare their performance.

## 1. Install and Import Dependencies

First, let's make sure all required packages are installed and import them.  
If you haven't installed Merlin yet, run:  
`pip install merlinquantum` in your terminal or `!pip install merlinquantum` in your notebook.

In [ ]:
import torch
import torch.nn as nn
import merlin as ML
from merlin.datasets import iris

## 2. Load and Prepare the Iris Dataset

We'll use the classic Iris dataset, a simple and well-known benchmark for classification.  
Let's load the data and convert it to PyTorch tensors for training.

In [ ]:
train_features, train_labels, train_metadata = iris.get_data_train()
test_features, test_labels, test_metadata = iris.get_data_test()

# Convert data to PyTorch tensors
X_train = torch.FloatTensor(train_features)
y_train = torch.LongTensor(train_labels)
X_test = torch.FloatTensor(test_features)
y_test = torch.LongTensor(test_labels)

print(f"Training samples: {X_train.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")
print(f"Features: {X_train.shape[1]}")
print(f"Classes: {len(torch.unique(y_train))}")

![iris](./img/iris.png)

## 3. Define the Hybrid Quantum-Classical Model

We'll build a neural network that combines classical and quantum layers:
- **Classical preprocessing**: Reduces the 4 input features to 3.
- **Quantum layer**: Processes the features using a quantum circuit.
- **Classical output**: Maps quantum outputs to the 3 Iris classes.

In [ ]:
class HybridIrisClassifier(nn.Module):
    """
    Hybrid model for Iris classification:
    - Classical layer reduces 4 features to 3
    - Quantum layer processes the 3 features
    - Classical output layer for 3-class classification
    """
    def __init__(self):
        super(HybridIrisClassifier, self).__init__()

        # Classical preprocessing layer: 4 → 8 → 3
        self.classical_in = nn.Sequential(
            nn.Linear(4, 8),
            nn.ReLU(),
            nn.Linear(8, 3),
            nn.Tanh()  # Normalize to [-1, 1] for quantum layer
        )

        # Quantum layer: processes 3 features
        self.quantum = ML.QuantumLayer.simple(
            input_size=3,
            n_params=number_of_quantum_params
        )

        # Classical output layer: quantum → 8 → 3
        self.classical_out = nn.Sequential(
            nn.Linear(self.quantum.output_size, 8),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(8, 3)
        )

        print(f"\nModel Architecture:")
        print(f"  Input: 4 features")
        print(f"  Classical preprocessing: 4 → 3")
        print(f"  Quantum layer: 3 → {self.quantum.output_size}")
        print(f"  Classical output: {self.quantum.output_size} → 3 classes")

    def forward(self, x):
        # Preprocess with classical layer
        x = self.classical_in(x)

        # Shift tanh output from [-1, 1] to [0, 1] for quantum layer
        x = (x + 1) / 2
        x = self.quantum(x)

        # Final classification
        x = self.classical_out(x)
        return x

## 4. Set Training and Quantum Parameters

You can adjust these parameters to see how they affect training and model performance.

In [ ]:
learning_rate = 0.05
number_of_epochs = 100
number_of_quantum_params = 200

## 5. Train the Hybrid Model

We'll train our hybrid model using standard PyTorch routines.  
The optimizer and loss function are set up as usual.

In [ ]:
# Create model
model = HybridIrisClassifier()

# Standard PyTorch training
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Training loop (standard PyTorch)
for epoch in range(number_of_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

## 6. Evaluate the Model

After training, let's evaluate our model on the test set and print the accuracy.

In [ ]:
# Evaluate on test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    predictions = torch.argmax(test_outputs, dim=1)
    accuracy = (predictions == y_test).float().mean().item()
    print(f"Test accuracy: {accuracy:.4f}")

# Conclusion

Congratulations! You've trained and evaluated a hybrid quantum-classical neural network using Merlin.  
Feel free to experiment with the model architecture, quantum parameters, or try other datasets!